In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchsummary import summary

import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

파라미터

In [2]:
epochs = 50
batch_size = 128
train_history = []
test_history = []
test_acc_history = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

데이터 로딩

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train = True,
                         download = True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081, ))
                         ])),
    batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train = False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081, ))
                         ])),
    batch_size = batch_size, shuffle=True)

In [4]:
train_loader.dataset[0][0].size() # 1차원의 28 * 28 이미지

torch.Size([1, 28, 28])

모델

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10) # 클래스 갯수 10
        
    def forward(self, x):

        x = self.conv1(x)
        
        x = F.max_pool2d(x,2,2)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.dropout2d(x, p = 0.5)
        
        x = F.max_pool2d(x,2,2)
        x = F.relu(x)
        
        x = x.view(-1, 20 * 4 * 4)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p = 0.5)
        
        x = self.fc2(x)
        
        return x

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.5)

In [7]:
summary(Net().to(device), input_size = (1, 28, 28), device="CPU")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
         MaxPool2d-2           [-1, 10, 12, 12]               0
            Conv2d-3             [-1, 20, 8, 8]           5,020
         Dropout2d-4             [-1, 20, 8, 8]               0
         MaxPool2d-5             [-1, 20, 4, 4]               0
            Linear-6                   [-1, 50]          16,050
         Dropout2d-7                   [-1, 50]               0
            Linear-8                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.08
Params size (MB): 0.08
Estimated Total Size (MB): 0.16
----------------------------------------------------------------


훈련

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    
    train_history.append(loss.item()) # 기록 저장
    print('Train [{}] \t Loss: {: .6f}'.format(epoch, loss.item()))

평가

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # 가장 높은 값을 가진 인덱스를 예측값으로
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        
        test_history.append(test_loss) # 기록 저장
        test_acc_history.append(test_accuracy)
        return test_loss, test_accuracy

모델 돌리기

In [10]:
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    if epoch%5 == 0:
        print('Test [{}] \t Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train [1] 	 Loss:  0.651814
Train [2] 	 Loss:  0.904685
Train [3] 	 Loss:  0.672427
Train [4] 	 Loss:  0.635635
Train [5] 	 Loss:  0.448784
Test [5] 	 Loss: 0.4962, Accuracy: 81.75%
Train [6] 	 Loss:  0.571998
Train [7] 	 Loss:  0.585119
Train [8] 	 Loss:  0.472354
Train [9] 	 Loss:  0.680685
Train [10] 	 Loss:  0.618920
Test [10] 	 Loss: 0.4170, Accuracy: 84.89%
Train [11] 	 Loss:  0.457761
Train [12] 	 Loss:  0.704669
Train [13] 	 Loss:  0.405635
Train [14] 	 Loss:  0.354832
Train [15] 	 Loss:  0.417170
Test [15] 	 Loss: 0.3785, Accuracy: 85.96%
Train [16] 	 Loss:  0.499188
Train [17] 	 Loss:  0.392053
Train [18] 	 Loss:  0.321424
Train [19] 	 Loss:  0.500928
Train [20] 	 Loss:  0.343255
Test [20] 	 Loss: 0.3519, Accuracy: 87.17%
Train [21] 	 Loss:  0.435528
Train [22] 	 Loss:  0.795976
Train [23] 	 Loss:  0.442781
Train [24] 	 Loss:  0.458791
Train [25] 	 Loss:  0.529157
Test [25] 	 Loss: 0.3439, Accuracy: 87.53%
Train [26] 	 Loss:  0.311349
Train [27] 	 Loss:  0.444105
Train [28] 	